# Photon ID Run 2 BDT classification

In [1]:
import uproot
import numpy as np
import pandas as pd
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

#import mplhep as hep
#hep.style.use("ATLAS")

import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay

import joblib

import seaborn as sns

In [2]:
datadir = "/home/chardong/y_identification/Venv/save_pkl/"
savedir = "/home/chardong/y_identification/work/save_plots/Py8_yj_jj_train_skim50/evt_mu/"
#datadir = "/eos/user/m/mdelmast/Data/EGamma/PhotonID/Run2/"
savedirmodel = "/home/chardong/y_identification/work/BDT_model/"

In [ ]:
#totald = pd.read_pickle(datadir+"Py8_yj_jj_mc16ade_pd122_train_w.pkl")
#totald = pd.read_pickle(datadir+"Py8_yj_jj_mc16ade_pd122_train_w_skim.pkl")
totald = pd.read_pickle(datadir+"Py8_yj_jj_mc16ade_pd122_train_w_skim_50.pkl")

In [ ]:
shower_shape_var = ['y_Reta',
                    'y_Rphi',
                    'y_weta2',
                    'y_fracs1',
                    'y_weta1',
                    'y_wtots1',
                    'y_Rhad',
                    'y_Rhad1',
                    'y_Eratio', 
                    'y_deltae']

prcpl_vx = ['evt_mu']       #Principal vertex parametrs, feel free to rocket me if I'm wrong


conv_var = [ 'y_convRadius', 'y_convType']

kinem_var = ['y_pt', 'y_eta', 'y_phi']

#truth_var = ['y_truth_pt', 'y_truth_eta', 'y_truth_pdgId', 'y_truth_mother_pdgId' ]
truth_var = ['y_truth_pt', 'y_truth_eta' ]

discriminating_var = shower_shape_var + kinem_var + conv_var + prcpl_vx
discriminating_var

### Prepare inputs for training

* `discriminating_var` containes the features used in the training
* Weights are added Y column to be able to access them after splitting in train and test samples.
* `test_size` represents the proportion of the dataset to include in the test split

In [ ]:
set(totald.columns)-set(discriminating_var)

In [ ]:
Y_var = ["truth_label",
         "weight",
         'y_IsTight',
         'y_IsLoose',
         'evt_mu', 
        ]

Y_var

### Save X and Y datasets with relevant variables

* Adding truth variables to X for performance studies, will be removed after splitting

In [ ]:
X = totald[discriminating_var+truth_var]
Y = totald[Y_var]

### Split dataset into train, validation and test samples

* Test dataset size: 20 %

In [ ]:
x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

### Split train + validation set into train and validation

* Train and validation are respectively 80% and 20% of 80% of total
* Save weights in separate arrays

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.2, random_state=42)

In [ ]:
weight_train = y_train["weight"]
weight_val   = y_val  ["weight"]
weight_test  = y_test ["weight"]

In [ ]:
othervars_train = y_train[['evt_mu', 'y_IsLoose', 'y_IsTight']]
othervars_val   = y_val  [['evt_mu', 'y_IsLoose', 'y_IsTight']]
othervars_test  = y_test [['evt_mu', 'y_IsLoose', 'y_IsTight']]

In [ ]:
truth_train = x_train[['y_truth_pt','y_truth_eta']]
truth_val   = x_val  [['y_truth_pt','y_truth_eta']]
truth_test  = x_test [['y_truth_pt','y_truth_eta']]

In [ ]:
truth_var_drop = list(set(y_train.columns)-{'truth_label'})
truth_var_drop

In [ ]:
y_train = y_train.drop(truth_var_drop, axis=1)
y_test  = y_test.drop(truth_var_drop, axis=1)
y_val   = y_val.drop(truth_var_drop, axis=1)

In [ ]:
x_train = x_train.drop(truth_var, axis=1)
x_test  = x_test.drop(truth_var, axis=1)
x_val   = x_val.drop(truth_var, axis=1)

In [ ]:
print('TRAINING   size = {:8d}'.format(len(y_train)))
print('TEST       size = {:8d}'.format(len(y_test)))
print('VALIDATION size = {:8d}'.format(len(y_val)))

In [ ]:
print('Number of signal events in test sample     :', len(y_test.query('truth_label == True')))
print('Number of background events in test sample :', len(y_test.query('truth_label == False')))

In [ ]:
y_train = np.ravel(y_train)
y_val = np.ravel(y_val)

### BDT training

In [ ]:
model = lgb.LGBMClassifier(learning_rate=0.05,  
                           num_leaves=35,
                           max_depth=-5,
                           objective='xentropy',
                           n_estimators=1000,
                           force_col_wise=True)

cb = [lgb.early_stopping(stopping_rounds=10),lgb.log_evaluation(30)]

In [ ]:
model.fit(x_train, y_train,
          sample_weight = weight_train,
          eval_set = [(x_train, y_train), (x_val, y_val)],  
          eval_names = ['Train', 'Validation'],
          eval_metric = 'xentropy', 
          eval_sample_weight = [weight_train, weight_val],
          callbacks=cb)

In [ ]:
 # Saving model to file
joblib.dump(model,'/home/chardong/y_identification/work/BDT_model/LGBMClassifier_model_hard_no_loose_lr0.05_35_skim30_Addevtmu.pkl')

In [ ]:
# Loading model from file
model_pickle = joblib.load(savedirmodel+"LGBMClassifier_model_hard_no_loose_lr0.05_35_skim30_Addevtmu.pkl")

### Feature importance

In [ ]:
# Cross-entropy evolution during training
lgb.plot_metric(model, figsize=(8,6))
plt.title('Metric during training')
plt.savefig(savedir+'metric_lr_0.09_35_skim50.pdf')

In [ ]:
# Feature importance: Numbers of times the feature is used in a model
lgb.plot_importance(model, importance_type='split', figsize=(8,6))
plt.title('Feature importance: split')
plt.savefig(savedir+'AddEvtmu_feature_split_lr_0.09_35_skim50.pdf')
plt.show()

In [ ]:
lgb.plot_importance(model, importance_type='gain', precision = None, figsize=(8,6))
plt.title('Feature importance: gain')
plt.savefig(savedir+'AddEvtmu_feature_gain_lr_0.09_35_skim50.pdf')
plt.show()

### Make predictions for test sample, add signal and background scores

In [ ]:
y_pred_prob_test = model.predict_proba(x_test)

df_pred_test = pd.DataFrame(y_pred_prob_test, columns=["background_score", "signal_score"])
df_pred_test.reset_index(inplace=True, drop=True)

In [ ]:
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True, drop=True)

truth_test.reset_index(inplace=True, drop=True)

othervars_test.reset_index(inplace=True, drop=True)

df_test_vars = x_test[kinem_var + conv_var]
df_test_vars.reset_index(inplace=True, drop=True)

weight_test = pd.DataFrame(weight_test)
weight_test.reset_index(inplace=True, drop=True)

In [ ]:
df_test = pd.concat([df_test_vars,
                     weight_test,
                     y_test,
                     othervars_test,
                     truth_test,
                     df_pred_test,
                    ], axis=1, join='inner', ignore_index=True)

In [ ]:
col_names = list(df_test_vars.columns) + \
            list(weight_test.columns) + \
            list(y_test.columns) + \
            list(othervars_test.columns) + \
            list(truth_test.columns) + \
            list(df_pred_test.columns)
df_test.columns = col_names

In [ ]:
df_test

In [ ]:
df_test.to_pickle(datadir+"test_sample_hard_scattering_skim50.pkl")

### BDT output

The BDT output has two columns: for each event a score (probability) to belong to class 0 or class 1  (here they are called `background_class` and `signal_class` ) is assigned.

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

plt.hist(df_test[(df_test["truth_label"]==0 )]["signal_score"],
         weights = df_test[(df_test["truth_label"]==0 )]["weight"], 
         bins = 50, log = True, density = True, alpha=0.5, 
         histtype = 'stepfilled', label='Background', color = 'b')

plt.hist(df_test[(df_test["truth_label"]==1 )]["signal_score"],
         weights = df_test[(df_test["truth_label"]==1 )]["weight"], 
         bins = 50, log = True, density = True, alpha=0.5,
         histtype = 'stepfilled', label='Signal', color = 'orange')

plt.ylabel('Frequency', fontsize = 14)

plt.legend(loc='upper center', fontsize = 14)
plt.gca().set(xlabel="LightGBM score (probability to be assigned to the signal class)")

plt.savefig(savedir+'AddEvtmu_score_lr_0.09_35_skim50.pdf')
plt.show()

### ROC curve

1) compute signal and background efficiencies for "official" cut-based Tight selection

In [ ]:
s_tot = sum( df_test[(df_test["truth_label"] == 1)]["weight"] )
s_selected = sum( df_test[(df_test["truth_label"] == 1) & (df_test['y_IsTight'].values)]["weight"] )
s_eff = s_selected / s_tot

b_tot = sum( df_test[(df_test["truth_label"] == 0)]["weight"] )
b_selected = sum( df_test[(df_test["truth_label"] == 0) & (df_test['y_IsTight'].values)]["weight"] )
b_eff = b_selected / b_tot

In [ ]:
print(f"Number of signal events                   = {s_tot:12.0f}")
print(f"Number of signal events passing Tight     = {s_selected:12.0f}")
print(f"Signal efficiency of cut-based Tight      = {100.*s_selected/s_tot:11.2f}%" )
print()
print(f"Number of background events               = {b_tot:12.0f}")
print(f"Number of background events passing Tight = {b_selected:12.0f}")
print(f"Background efficiency of cut-based Tight  = {100.*b_selected/b_tot:11.2f}%" )
print(f"Background rejection of cut-based Tight   = {100.*(1-b_selected/b_tot):11.2f}%" )

2) Plot ROC curve of trained BDT with weighted events, compare to current cut-based Tight selection 

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

mod_disp = RocCurveDisplay.from_estimator(model, x_test, y_test, sample_weight=weight_test, 
                                          label="BDT", ax=ax) 

plt.plot(b_eff, s_eff, marker="x", markersize=10, color="red", label = 'Cut-based Tight ID')

plt.xlabel('Background efficiency')
plt.ylabel('Signal efficiency')

#plt.xlim([0.0, 0.15])
#plt.ylim([0.65, 1.0])
#plt.title('BDT ROC curve - zoom')

plt.title('BDT ROC curve')
plt.legend()

plt.savefig(savedir+'AddEvtmu_ROC_lr_0.05_35_weight_skim50.pdf')
plt.show()

### Correlation between input variables

In [ ]:
#x_train_sel = x_train.query('y_wtots1 >- 800 & y_weta1 > -800')
#x_train_sel.reset_index(inplace=True, drop=True)

In [ ]:
corr = x_train.corr()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.rcParams.update({'font.size': 10})

sns.heatmap(corr, vmin=-1, vmax=1, cmap="coolwarm", ax=ax, annot=True, fmt=".2f")

plt.tight_layout()
plt.savefig(savedir+'AddEvtmu_correlations_train_sample_all_skim50.pdf')
plt.show()

In [ ]:
#corr_sig = x_train[(y_train==1)].corr()
#corr_bkg = x_train[(y_train==0)].corr()